In [29]:
import pandas as pd

In [30]:
artists = pd.read_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Spotify\latest_artists_with_infos.parquet")
artists_null = artists[artists['Spotify Name'].isnull()]
artists_null

,Artist,Spotify Name,Genres,Popularity
100,Vulfpeck,None,None,NaN
101,Negative FX & Last Rights,None,None,NaN
102,The Mighty Diamonds,None,None,NaN
103,"Phineas Newborn, Jr.",None,None,NaN
104,Lightnin' Hopkins,None,None,NaN
...,...,...,...,...
9315,No Neck Blues Band With John Fahey And Coach F...,None,None,NaN
11802,"Deonis ""Pumah"" Cook",None,None,NaN
11918,The Asteroid #4,None,None,NaN
12096,"Roosevelt ""Booba"" Barnes",None,None,NaN


In [31]:
lost_artists = artists_null['Artist'].tolist()
lost_artists

['Vulfpeck',
 'Negative FX & Last Rights',
 'The Mighty Diamonds',
 'Phineas Newborn, Jr.',
 "Lightnin' Hopkins",
 'The New Deal',
 'LOC',
 'Phyllis Hyman',
 'Marcus Roberts',
 'Walter "Wolfman" Washington',
 'Iain macDonald',
 'The Locust',
 'Dinosaur Eyelids',
 'Mac Wiseman',
 'Murs',
 'Down To The Bone featuring Roy Ayers',
 'Melt Banana',
 'The Sensational Alex Harvey Band',
 'Horse Thief',
 'Bon Jovi',
 'Obray Ramsey',
 'The Flaming Lips',
 'Dayna Stephens',
 'Gustav Mahler',
 'Vertical Horizon',
 'Four Tet',
 'Faith Hill',
 'Clarence Gatemouth Brown',
 'Louis Armstrong',
 'Aurora',
 'Jellyfish',
 'Giorgio Moroder',
 'Kirk Franklin',
 'Ce Ce Peniston',
 'Sixpence None the Richer',
 'Roberta Gambarini',
 'Koffee Brown Featuring Lady Luck',
 'Wisin & Yandel',
 'Big Jack Johnson & The Oilers',
 'Gin Blossoms',
 'Clement "Coxsone" Dodd',
 'Jazz Gillum',
 'Christina Aguilera',
 'Witchcraft',
 'Pro-Pain',
 'Dub Pistols',
 'Unsane',
 'Adam Arcuragi',
 'Supersystem',
 'Os Mutantes',
 'Gwe

In [32]:
import os
import base64
import requests
from requests import post, get
import json
from tqdm import tqdm
import pandas as pd
import time

def get_token():
    client_id = '7727b6a6602f4dcf99d337ee11f0983e'
    client_secret = '6b4958253cb544aaae1b0bf4858726ee'
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    json_result = json.loads(response.content)

    if "access_token" in json_result:
        token = json_result["access_token"]
        return token
    else:
        print("Error: Failed to get access token")
        print(json_result)
        return None

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_artist(access_token, artist_name):
    search_url = 'https://api.spotify.com/v1/search'
    search_params = {'q': artist_name, 'type': 'artist', 'limit': 1}
    search_headers = {'Authorization': f'Bearer {access_token}'}

    response = requests.get(search_url, params=search_params, headers=search_headers)
    response.raise_for_status()
    items = response.json()['artists']['items']
    
    if not items:
        return None

    artist = items[0]
    return {
        'Artist': artist_name,
        'Spotify Name': artist['name'],
        'Genres': artist['genres'],
        'Popularity': artist['popularity'],
        'Spotify ID': artist['id']
    }


# token = get_token()

# df = artists_null

# # df = df[:10]

# df["Spotify Name"] = None
# df["Spotify ID"] = None
# df["Genres"] = None
# df["Popularity"] = None

# with tqdm(total=len(df)) as pbar:
#     for i, artist_name in enumerate(df["Artist"]):
#         pbar.set_description(f"Processing artist {i+1} of {len(df)}: {artist_name}")
#         # token = get_token()
#         if token:
#             spotifyname, spotifyid, genre, pop = get_artist_info(token, artist_name)
#             if spotifyname:
#               df.at[i, "Spotify Name"] = spotifyname
#             else:
#                 df.at[i, "Spotify Name"] = ""
#             if spotifyid:
#               df.at[i, "Spotify ID"] = spotifyid
#             else:
#                 df.at[i, "Spotify ID"] = ""
#             if genre:
#               df.at[i, "Genres"] = genre
#             else:
#                 df.at[i, "Genres"] = ""
#             if pop:
#               df.at[i, "Popularity"] = pop

#         pbar.update(1)

# # df.head(10)
# df.to_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Spotify\lost_artists_with_infos.parquet")

In [34]:
token = get_token()
spotify_info = []
for artist in artists_null['Artist']:
    artist_info = search_artist(token, artist)
    if artist_info is not None:
        spotify_info.append(artist_info)
    else:
        print(f"No artist information found for {artist}")
    time.sleep(0.5)

filtered_spotify_info = [info for info in spotify_info if info is not None]
spotify_df = pd.DataFrame(filtered_spotify_info)
spotify_df

No artist information found for Clement "Coxsone" Dodd
No artist information found for Scientist / Jammy / The Roots Radics
No artist information found for No Neck Blues Band With John Fahey And Coach Fingers
No artist information found for Deonis "Pumah" Cook
No artist information found for Roosevelt "Booba" Barnes


,Artist,Spotify Name,Genres,Popularity,Spotify ID
0,Vulfpeck,Vulfpeck,"[ann arbor indie, funk rock, instrumental funk]",63,7pXu47GoqSYRajmBCjxdD6
1,Negative FX & Last Rights,Negative FX & Last Rights,[],0,58xJSnOYaZqkxqNak2MLJR
2,The Mighty Diamonds,Mighty Diamonds,"[dub, lovers rock, old school dancehall, regga...",47,2MvXKrHantfY5wyPiCOo2k
3,"Phineas Newborn, Jr.",Phineas Newborn Jr.,"[bebop, hard bop, jazz piano, stride]",29,1WsMRWV5KEC2AxpYkeb2Cf
4,Lightnin' Hopkins,Lightnin' Hopkins,"[acoustic blues, blues, country blues, delta b...",47,6EZzVXM2uDRPmnHWq9yPDE
...,...,...,...,...,...
673,Nas ft. Lofey,"Nas & Damian ""Jr. Gong"" Marley",[reggae fusion],50,3w2WYNyPCPkZFoC1EqfCMe
674,Phil Fuldner Works #2,Phil Will Goode,[],40,6RaRi6B6e28yTBdTvu1Gzy
675,Yellow #5,Yellow #5,[],1,0YebsbrtgAbqMPdpWFCPYn
676,The Asteroid #4,The Asteroid #4,[psych gaze],2,4jp2FOUJpmfpT79a6rz82x


In [35]:
spotify_df.to_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Spotify\lost_artists_with_infos.parquet")

In [37]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

def search(artist_list, checkpoint_interval=1000):
    artist_relations = {}
    searched_artist_names = {}
    checkpoint_dir = 'lost_artists_checkpoints'
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_file = os.path.join(checkpoint_dir, 'checkpoint.pkl')
    start_index = 0

    if os.path.exists(checkpoint_file):
        checkpoint = pd.read_pickle(checkpoint_file)
        artist_relations = checkpoint['artist_relations']
        searched_artist_names = checkpoint['searched_artist_names']
        start_index = checkpoint['index']
        print(f"Resuming from checkpoint: {start_index}")

    for i, a in tqdm(enumerate(artist_list[start_index:]), total=len(artist_list[start_index:]), position=0, desc="Processing artists"):
        index = i + start_index

        aa = a.replace(" ", "+")
        url = f'https://inflooenz.com/?artist={aa}&submit=Search'
        try:
            page = requests.get(url)
            page.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error: {a} not found")
            continue

        soup = BeautifulSoup(page.content, 'html.parser')

        # Extract the searched artist name
        artist_name_tag = soup.find('h1')
        searched_artist_name = None
        if artist_name_tag:
            searched_artist_name_tag = artist_name_tag.find('span', {'class': 'artist', 'itemprop': 'name'})
            if searched_artist_name_tag:
                searched_artist_name = searched_artist_name_tag.text

        followers = []
        influencers = []

        followers_list = soup.find('ul', attrs={'class':'influences-list', 'id':'followers-list'})
        if followers_list:
            for follower in followers_list.find_all('li')[:-1]:
                if follower.text != "":
                    followers.append(follower.text)
        influencers_list = soup.find('ul', attrs={'class':'influences-list', 'id':'influencers-list'})
        if influencers_list:
            for influencer in influencers_list.find_all('li')[:-1]:
                if influencer.text != "":
                    influencers.append(influencer.text)

        if searched_artist_name:
            artist_relations[a] = {'followers': followers, 'influencers': influencers}
            searched_artist_names[a] = searched_artist_name
        else:
            artist_relations[a] = {'followers': followers, 'influencers': influencers}
            searched_artist_names[a] = a

        if index > 0 and index % checkpoint_interval == 0:
            checkpoint = {'artist_relations': artist_relations, 'searched_artist_names': searched_artist_names, 'index': index}
            pd.to_pickle(checkpoint, checkpoint_file)

    checkpoint = {'artist_relations': artist_relations, 'searched_artist_names': searched_artist_names, 'index': index}
    pd.to_pickle(checkpoint, checkpoint_file)

    return artist_relations, searched_artist_names

In [38]:
artist_list = spotify_df['Spotify Name'].tolist()

artist_relations, searched_artist_names = search(artist_list, checkpoint_interval=10)

# # Adding followers and influencers columns to the artists_df DataFrame
spotify_df['followers'] = spotify_df['Spotify Name'].apply(lambda x: artist_relations[x]['followers'] if x in artist_relations else [])
spotify_df['influencers'] = spotify_df['Spotify Name'].apply(lambda x: artist_relations[x]['influencers'] if x in artist_relations else [])
spotify_df['searched_artist_name'] = spotify_df['Spotify Name'].apply(lambda x: searched_artist_names[x] if x in searched_artist_names else x)
spotify_df.to_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Spotify\lost_artists_with_relations.parquet")

Processing artists: 100%|██████████| 678/678 [30:42<00:00,  2.72s/it]  


In [39]:
spotify_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Vulfpeck,Vulfpeck,"[ann arbor indie, funk rock, instrumental funk]",63,7pXu47GoqSYRajmBCjxdD6,[],"[Booker T. & the MG's, The Funk Brothers, The ...",Vulfpeck
1,Negative FX & Last Rights,Negative FX & Last Rights,[],0,58xJSnOYaZqkxqNak2MLJR,[Dropkick Murphys],[],Negative FX
2,The Mighty Diamonds,Mighty Diamonds,"[dub, lovers rock, old school dancehall, regga...",47,2MvXKrHantfY5wyPiCOo2k,[],[],Mighty Diamonds
3,"Phineas Newborn, Jr.",Phineas Newborn Jr.,"[bebop, hard bop, jazz piano, stride]",29,1WsMRWV5KEC2AxpYkeb2Cf,[],[],Phineas Newborn Jr.
4,Lightnin' Hopkins,Lightnin' Hopkins,"[acoustic blues, blues, country blues, delta b...",47,6EZzVXM2uDRPmnHWq9yPDE,"[Quicksilver Messenger Service, Spirit, Albert...","[Mississippi Fred McDowell, Big Bill Broonzy, ...",Lightnin' Hopkins
...,...,...,...,...,...,...,...,...
673,Nas ft. Lofey,"Nas & Damian ""Jr. Gong"" Marley",[reggae fusion],50,3w2WYNyPCPkZFoC1EqfCMe,"[Joe Budden, Kanye West, 50 Cent, Roc Marciano...","[A Tribe Called Quest, LL Cool J, Gang Starr, ...",Nas
674,Phil Fuldner Works #2,Phil Will Goode,[],40,6RaRi6B6e28yTBdTvu1Gzy,[],[],Phil Will Goode
675,Yellow #5,Yellow #5,[],1,0YebsbrtgAbqMPdpWFCPYn,[],[],Yellow Claw
676,The Asteroid #4,The Asteroid #4,[psych gaze],2,4jp2FOUJpmfpT79a6rz82x,[],[],The Asteroids Galaxy Tour


In [41]:
spotify_df = spotify_df.loc[~(spotify_df['followers'].apply(len) == 0) | ~(spotify_df['influencers'].apply(len) == 0)]

In [42]:
spotify_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Vulfpeck,Vulfpeck,"[ann arbor indie, funk rock, instrumental funk]",63,7pXu47GoqSYRajmBCjxdD6,[],"[Booker T. & the MG's, The Funk Brothers, The ...",Vulfpeck
1,Negative FX & Last Rights,Negative FX & Last Rights,[],0,58xJSnOYaZqkxqNak2MLJR,[Dropkick Murphys],[],Negative FX
4,Lightnin' Hopkins,Lightnin' Hopkins,"[acoustic blues, blues, country blues, delta b...",47,6EZzVXM2uDRPmnHWq9yPDE,"[Quicksilver Messenger Service, Spirit, Albert...","[Mississippi Fred McDowell, Big Bill Broonzy, ...",Lightnin' Hopkins
5,The New Deal,The New Deal,"[jam band, jamtronica, livetronica]",26,13Qw23XBRFmgTsU6hI8uVK,[],"[808 State, Coldcut, Jamiroquai, Masters at Wo...",The New Deal
6,LOC,Local Natives,"[chillwave, indie pop, indie rock, la indie, m...",59,75dQReiBOHN37fQgWQrIAJ,[],"[Crosby, Stills & Nash, The Shins, Animal Coll...",Local Natives
...,...,...,...,...,...,...,...,...
668,Prick,Prick,"[industrial metal, industrial rock]",22,4hILJw5uchW1ZZJO10icIz,[],[Nine Inch Nails],Prick
669,Aretha Franklin Duet with Luther Vandross,Aretha Franklin,"[adult standards, classic soul, jazz blues, me...",72,7nwUJBm0HE4ZxD3f5cy5ok,"[Beyoncé, Donna Summer, Mary J. Blige, Natalie...","[Billie Holiday, Clara Ward, Dinah Washington,...",Aretha Franklin
671,Windsor For The Derby,Windsor For The Derby,[tampa indie],23,1BlsomJZZh7jhA7IvgS1Al,[],"[Yo La Tengo, Faust, Neu!]",Windsor For The Derby
672,Little Matador,Little Matador,[belfast indie],2,6FeebWsBMCP5p9Ojoa2hGs,[],"[Nirvana, Mercury Rev, Led Zeppelin, The Clash...",Little Matador


In [48]:
cur_artists = pd.read_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Results\filtered_artists_with_relations.parquet")

In [49]:
cur_artists_v = set(cur_artists['Spotify Name'])
spotify_df_v = set(spotify_df['Spotify Name'])

In [50]:
common_values = spotify_df_v.intersection(cur_artists_v)
unique_values = spotify_df_v.symmetric_difference(cur_artists_v)

In [51]:
print(len(common_values))
print(len(unique_values))

116
9503


In [52]:
common_values

{'Afrika Bambaataa',
 'Albert Ayler',
 'Amon Tobin',
 'Anthrax',
 'Aretha Franklin',
 'Armin van Buuren',
 'Band of Horses',
 'Bert Kaempfert',
 'Big Sandy & His Fly-Rite Boys',
 'Billie Eilish',
 'Billy May',
 'Black Star',
 'Blue Öyster Cult',
 'Bright Eyes',
 'Brigitte Bardot',
 'Buddy Holly',
 'Camila Cabello',
 'Cecilia Bartoli',
 'Clarence "Gatemouth" Brown',
 'DJ Krush',
 'Daddy Yankee',
 'Danger Mouse',
 'Dave Matthews Band',
 'Death Cab for Cutie',
 'Dirty Projectors',
 'Dizzy Gillespie',
 'Dub Pistols',
 'Eddie Henderson',
 'Elis Regina',
 'Eminem',
 'Eugenia Leon',
 'Fiona Apple',
 'Florence + The Machine',
 'Frédéric Chopin',
 'Gladys Knight & The Pips',
 'Gossip',
 'Gregory Isaacs',
 'Guilty Simpson',
 'Hearts and Flowers',
 'Hypocrisy',
 'INXS',
 'Ike Turner',
 'Ill Niño',
 'Immortal Technique',
 'James Brown',
 'Jimmie Rodgers',
 'John Zorn',
 'Johnny Flynn',
 'Johnny Thunders',
 'Joshua Radin',
 'Keith Jarrett',
 'King Diamond',
 'Kirk Franklin',
 'Kool & The Gang',
 'L

In [61]:
unique_spotify_df = spotify_df[~spotify_df['Spotify Name'].isin(common_values)]

In [62]:
unique_spotify_df[unique_spotify_df['Spotify Name'] == 'B.B. King']

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
651,B.B. King,B.B. King,"[blues, blues rock, classic rock, electric blu...",63,5xLSa7l4IV1gsQfhAMvl0U,"[John Scofield, Bonnie Raitt, Buddy Guy, Eddy ...","[Charlie Christian, Django Reinhardt, Lonnie J...",B.B. King


In [63]:
unique_spotify_df = unique_spotify_df.reset_index(drop=True)
unique_spotify_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Vulfpeck,Vulfpeck,"[ann arbor indie, funk rock, instrumental funk]",63,7pXu47GoqSYRajmBCjxdD6,[],"[Booker T. & the MG's, The Funk Brothers, The ...",Vulfpeck
1,Negative FX & Last Rights,Negative FX & Last Rights,[],0,58xJSnOYaZqkxqNak2MLJR,[Dropkick Murphys],[],Negative FX
2,Phyllis Hyman,Phyllis Hyman,"[classic soul, disco, funk, neo soul, philly s...",43,3oqSDVaf6RoBGreqOlgUpv,"[Toni Braxton, Regina Belle]",[],Phyllis Hyman
3,Marcus Roberts,Marcus Roberts,"[jazz piano, stride]",22,6dz608P8sHylVvVVo5OLx2,[],"[Thelonious Monk, Ahmad Jamal, James P. Johnso...",Marcus Roberts
4,The Locust,The Locust,"[cybergrind, mathcore, nintendocore, noisecore...",27,6uZOWljC7E3k7ZjVNKwZyW,[],"[Drive Like Jehu, Boredoms]",The Locust
...,...,...,...,...,...,...,...,...
446,Mint Condition,Mint Condition,"[contemporary r&b, funk, hip pop, minneapolis ...",52,4MT97VeycuegSicrrphisJ,[],"[Stevie Wonder with B.B. King, The Bar-Kays]",Mint Condition
447,Prick,Prick,"[industrial metal, industrial rock]",22,4hILJw5uchW1ZZJO10icIz,[],[Nine Inch Nails],Prick
448,Windsor For The Derby,Windsor For The Derby,[tampa indie],23,1BlsomJZZh7jhA7IvgS1Al,[],"[Yo La Tengo, Faust, Neu!]",Windsor For The Derby
449,Little Matador,Little Matador,[belfast indie],2,6FeebWsBMCP5p9Ojoa2hGs,[],"[Nirvana, Mercury Rev, Led Zeppelin, The Clash...",Little Matador


In [69]:
out1 = unique_spotify_df[['Spotify Name', 'searched_artist_name']]
out1

,Spotify Name,searched_artist_name
0,Vulfpeck,Vulfpeck
1,Negative FX & Last Rights,Negative FX
2,Phyllis Hyman,Phyllis Hyman
3,Marcus Roberts,Marcus Roberts
4,The Locust,The Locust
...,...,...
446,Mint Condition,Mint Condition
447,Prick,Prick
448,Windsor For The Derby,Windsor For The Derby
449,Little Matador,Little Matador


In [70]:
out1.to_csv('out1.csv')

In [71]:
unique_spotify_df.to_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Spotify\unique_lost_artists_with_relations.parquet")

In [72]:
pre_spotify_df = pd.read_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Results\filtered_artists_with_relations.parquet")

In [73]:
pre_spotify_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses
...,...,...,...,...,...,...,...,...
9117,Diana Ross and the Supremes,Diana Ross & The Supremes,"[classic girl group, motown, soul]",51.0,3MdG05syQeRYPPcClLaUGl,"[Beyoncé, Mary J. Blige, Brenda Holloway, Donn...","[The Temptations, Billie Holiday, Ella Fitzger...",Diana Ross
9128,Frank Black Francis,Frank Black & The Catholics,[alternative rock],30.0,5Q8gM5SMBT2LAoZ5LzqAvH,"[Eddie Vedder, Dave Grohl, Veruca Salt, Superc...","[The Beach Boys, David Bowie, Hüsker Dü, Iggy ...",Frank Black
9184,Louis Armstrong - Itinéraire d'un génie,Louis Armstrong & His Hot Five,"[hot jazz, new orleans jazz]",31.0,19eLuQmk9aCobbVDHc6eek,"[George Benson, Miles Davis, Charlie Parker, D...","[King Oliver, Bunk Johnson, Kid Ory, Fletcher ...",Louis Armstrong
9454,Steve Harley,Steve Harley,"[classic uk pop, glam rock, pub rock]",50.0,3dS8rLINyM7EYuMXryXJym,[],[The Pretty Things],Steve Harley & Cockney Rebel


In [75]:
res_df = pd.concat([pre_spotify_df, unique_spotify_df])
res_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses
...,...,...,...,...,...,...,...,...
446,Mint Condition,Mint Condition,"[contemporary r&b, funk, hip pop, minneapolis ...",52.0,4MT97VeycuegSicrrphisJ,[],"[Stevie Wonder with B.B. King, The Bar-Kays]",Mint Condition
447,Prick,Prick,"[industrial metal, industrial rock]",22.0,4hILJw5uchW1ZZJO10icIz,[],[Nine Inch Nails],Prick
448,Windsor For The Derby,Windsor For The Derby,[tampa indie],23.0,1BlsomJZZh7jhA7IvgS1Al,[],"[Yo La Tengo, Faust, Neu!]",Windsor For The Derby
449,Little Matador,Little Matador,[belfast indie],2.0,6FeebWsBMCP5p9Ojoa2hGs,[],"[Nirvana, Mercury Rev, Led Zeppelin, The Clash...",Little Matador


In [77]:
res_df = res_df.reset_index(drop=True)
res_df

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses
...,...,...,...,...,...,...,...,...
9615,Mint Condition,Mint Condition,"[contemporary r&b, funk, hip pop, minneapolis ...",52.0,4MT97VeycuegSicrrphisJ,[],"[Stevie Wonder with B.B. King, The Bar-Kays]",Mint Condition
9616,Prick,Prick,"[industrial metal, industrial rock]",22.0,4hILJw5uchW1ZZJO10icIz,[],[Nine Inch Nails],Prick
9617,Windsor For The Derby,Windsor For The Derby,[tampa indie],23.0,1BlsomJZZh7jhA7IvgS1Al,[],"[Yo La Tengo, Faust, Neu!]",Windsor For The Derby
9618,Little Matador,Little Matador,[belfast indie],2.0,6FeebWsBMCP5p9Ojoa2hGs,[],"[Nirvana, Mercury Rev, Led Zeppelin, The Clash...",Little Matador


In [80]:
df_filtered = res_df.drop_duplicates(subset='searched_artist_name', keep='first')
df_filtered

,Artist,Spotify Name,Genres,Popularity,Spotify ID,followers,influencers,searched_artist_name
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M,"[Alice in Chains, Ween, Iron Maiden, Pantera, ...","[Sex Pistols, The Yardbirds, Ramones, The Stoo...",Motörhead
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG,[],"[New Edition, Jodeci, Guy]",Color Me Badd
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx,"[No Age, The Fiery Furnaces]","[Sonic Youth, The Plastic People of the Univer...",Trumans Water
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs,[],[Steve Forbert],Cliff Eberhardt
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8,[],"[Blondie, Battant]",The Waitresses
...,...,...,...,...,...,...,...,...
9614,Orange Juice,Orange Juice,"[art rock, c86, new wave, post-punk, scottish ...",42.0,2u7xZP39dtED9EuRX9MUwu,"[Teenage Fanclub, Withered Hand, Franz Ferdina...",[Hot Chocolate],Orange Juice
9615,Mint Condition,Mint Condition,"[contemporary r&b, funk, hip pop, minneapolis ...",52.0,4MT97VeycuegSicrrphisJ,[],"[Stevie Wonder with B.B. King, The Bar-Kays]",Mint Condition
9616,Prick,Prick,"[industrial metal, industrial rock]",22.0,4hILJw5uchW1ZZJO10icIz,[],[Nine Inch Nails],Prick
9617,Windsor For The Derby,Windsor For The Derby,[tampa indie],23.0,1BlsomJZZh7jhA7IvgS1Al,[],"[Yo La Tengo, Faust, Neu!]",Windsor For The Derby


In [82]:
df_filtered.to_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Results\April_latest_artists_with_relations.parquet")